In [1]:
import glob
import numpy as np
import ROOT
import random
from scipy import interpolate, optimize
import pickle

Welcome to JupyROOT 6.26/04


In [2]:
filesER = glob.glob("/home/smalu/Documents/NSC_veto_ER_NR/Pulse/*ER*.txt")
filesNR = glob.glob("/home/smalu/Documents/NSC_veto_ER_NR/Pulse/*NR*.txt")

In [3]:
%jsroot on

In [4]:
#writing pulses into arrays
number_ER = len(filesER)
number_NR = len(filesNR)
pulsesER = {}
primaryPulsesER = []
energiesER = []
pchannelER = []
counter = 0
for index, file in enumerate(filesER):
    pulse = {}
    #print(file)
    #print(round(float(file.split('/')[6].split('_')[1][:-3]), 1))
    energy = round(float(file.split('/')[6].split('_')[1][:-3]), 1)
    for i in range(4):
        pulse[i] = []
    with open(file) as f:
        for line in f:
            for i in range(4):
                pulse[i].append(float(line.split()[i]))
    primaryCh = np.argmax([max(pulse[0]),max(pulse[1]),max(pulse[2]),max(pulse[3])])
    if energy>1:
        pulsesER[counter] = pulse
        primaryPulsesER.append(pulse[primaryCh])
        energiesER.append(energy)
        pchannelER.append(primaryCh)
        counter+=1
        
pulsesNR = {}
primaryPulsesNR = []
energiesNR = []
pchannelNR = []
counter = 0
for index, file in enumerate(filesNR):
    pulse = {}
    energy = round(float(file.split('/')[6].split('_')[1][:-3]), 1)
    for i in range(4):
        pulse[i] = []
    with open(file) as f:
        for line in f:
            for i in range(4):
                pulse[i].append(float(line.split()[i]))
    primaryCh = np.argmax([max(pulse[0]),max(pulse[1]),max(pulse[2]),max(pulse[3])])
    if energy>0.7:
        pulsesNR[counter] = pulse
        primaryPulsesNR.append(pulse[primaryCh])
        energiesNR.append(energy)
        pchannelNR.append(primaryCh)
        counter+=1

In [5]:
nSamples = len(pulsesER[0][0])
print("Number of ER pulses: "+str(len(energiesER)))
print("Number of NR pulses: "+str(len(energiesNR)))

Number of ER pulses: 217
Number of NR pulses: 62


In [6]:
#plotting some of the ER pulses (All Channes)
#testing purposes

cPulses= ROOT.TCanvas("cPulses", "", 1200, 300)
cPulses.Divide(4,1)
channels = ['A', 'B', 'C', 'D']
colors = [ROOT.kRed, ROOT.kBlue, ROOT.kGreen]
nSamples = len(pulsesER[0][0])
pulseGraphs, legend1 = {}, {}
for k in range(4):
    n = random.randint(0, len(pulsesER))
    pulseGraphs[k] = {}
    X = [float(i) for i in range(nSamples)]
    X = np.array(X)
    for i in range(4):
        Y = np.array(pulsesER[n][i])
        Y = Y/max(Y)
        pulseGraphs[k][i] = ROOT.TGraph(nSamples,X, Y)


    minX = 250
    maxX = 350
    legend1[k] = ROOT.TLegend(0.8,0.7,0.9,0.9)
    for i in range(3):
        cPulses.cd(k+1)

        pulseGraphs[k][i+1].SetLineColor(colors[i])
        pulseGraphs[k][i+1].SetMarkerColor(colors[i])
        pulseGraphs[k][i+1].SetLineWidth(2)
        pulseGraphs[k][i+1].GetXaxis().SetRangeUser(minX, maxX)
        #pulseGraphs[k][i+1].GetYaxis().SetRangeUser(-.1e-6, 0.7e-6)
        pulseGraphs[k][i+1].GetXaxis().SetTitle('us')
        pulseGraphs[k][i+1].GetYaxis().SetTitle('a.u')
        pulseGraphs[k][i+1].GetYaxis().SetTitleOffset(1.4)
        pulseGraphs[k][i+1].SetTitle(str(energiesER[n])+" keV")
        if i == 0:
            pulseGraphs[k][i+1].Draw()
        else:
            pulseGraphs[k][i+1].Draw("SAME")

        legend1[k].AddEntry(pulseGraphs[k][i+1],channels[i+1],"l")

    legend1[k].Draw("SAME")
cPulses.Draw()

In [7]:
#function calculates rise timestamp given the pulse and the percentage
def calcRise(pulse, pct):
    pmax = max(pulse)
    maxbin = np.argmax(pulse)
    result = 0
    time = [float(i) for i in range(nSamples)]
    time = np.array(time)
    for i in range(int(nSamples / 4)):
        if pulse[maxbin - i] < pct * pmax and maxbin - i > 0:
            y = np.array(
                  [pulse[maxbin - i - 1], pulse[maxbin - i], pulse[maxbin - i + 1], pulse[maxbin - i + 2]])
            x = np.array(
                  [time[maxbin - i - 1], time[maxbin - i], time[maxbin - i + 1], time[maxbin - i + 2]])
            f1 = interpolate.interp1d(x, y, kind='linear')
            g = lambda x: f1(x) - pct * pmax
            try:
                result = optimize.newton(g, time[maxbin - i])
            except (ValueError, RuntimeError):
                result = 0
            break
    return result

In [8]:
#write timing info into pickle for further analysis
input_arr_ER, input_arr_NR = [], []
for i in range(len(energiesER)):
    arr_ER = np.zeros(10, dtype = np.float64)
    arr_NR = np.zeros(10, dtype = np.float64)
    for j in range(8):
        arr_ER[j] = calcRise(primaryPulsesER[i], (j+2)*.1)-calcRise(primaryPulsesER[i], (j+1)*.1)
        arr_ER[8] = energiesER[i]
        arr_ER[9] = 0
    input_arr_ER.append(arr_ER)
    
for i in range(len(energiesNR)):
    for j in range(8):
        arr_NR[j] = calcRise(primaryPulsesNR[i], (j+2)*.1)-calcRise(primaryPulsesNR[i], (j+1)*.1)
        arr_NR[8] = energiesNR[i]
        arr_NR[9] = 1
    input_arr_NR.append(arr_NR)
    
with open('timeDiff_ER.pickle', 'wb') as handle:
    pickle.dump(input_arr_ER,handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('timeDiff_NR.pickle', 'wb') as handle1:
    pickle.dump(input_arr_NR,handle1, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
#calculating rise timestamps
primaryRiseER10 = []
primaryRiseER40 = []
primaryRiseER90 = []
for i in range(len(primaryPulsesER)):
    primaryRiseER10.append(calcRise(primaryPulsesER[i],.1))
    primaryRiseER40.append(calcRise(primaryPulsesER[i],.4))
    primaryRiseER90.append(calcRise(primaryPulsesER[i],.9))
    
primaryRiseNR10 = []
primaryRiseNR40 = []
primaryRiseNR90 = []
for i in range(len(primaryPulsesNR)):
    primaryRiseNR10.append(calcRise(primaryPulsesNR[i],.1))
    primaryRiseNR40.append(calcRise(primaryPulsesNR[i],.4))
    primaryRiseNR90.append(calcRise(primaryPulsesNR[i],.9))

In [10]:
#Histograms with rise time distribution
nbins = 25
riseER1 = ROOT.TH1F("riseER1", "", nbins, 0.0, 15)
riseNR1 = ROOT.TH1F("riseNR1", "", nbins, 0.0, 15)
riseER = ROOT.TH1F("riseER", "", nbins, 0.0, 15)
riseNR = ROOT.TH1F("riseNR", "", nbins, 0.0, 15)
riseER1_40 = ROOT.TH1F("riseER1_40", "", nbins, 1, 5)
riseNR1_40 = ROOT.TH1F("riseNR1_40", "", nbins, 1, 5)
riseER_40 = ROOT.TH1F("riseER_40", "", nbins, 1, 5)
riseNR_40 = ROOT.TH1F("riseNR_40", "", nbins, 1, 5)
rise90_vs_energy = ROOT.TH2F("rise90_vs_energy", "", nbins, 1, 5, nbins, 0.0, 15)
rise90_vs_energy1 = ROOT.TH2F("rise90_vs_energy1", "", nbins, 1, 5, nbins, 0.0, 15)
rise40_vs_energy = ROOT.TH2F("rise40_vs_energy", "", nbins, 1, 5, nbins, 0.0, 15)
rise40_vs_energy1 = ROOT.TH2F("rise40_vs_energy1", "", nbins, 1, 5, nbins, 0.0, 15)

rise90B_vs_energy = ROOT.TH2F("rise90B_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)
rise90C_vs_energy = ROOT.TH2F("rise90C_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)
rise90D_vs_energy = ROOT.TH2F("rise90D_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)
rise40B_vs_energy = ROOT.TH2F("rise40B_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)
rise40C_vs_energy = ROOT.TH2F("rise40C_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)
rise40D_vs_energy = ROOT.TH2F("rise40D_vs_energy", "", nbins, 1, 3, nbins, 0.0, 20)

rise90B_vs_amp = ROOT.TH2F("rise90B_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)
rise90C_vs_amp = ROOT.TH2F("rise90C_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)
rise90D_vs_amp = ROOT.TH2F("rise90D_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)
rise40B_vs_amp = ROOT.TH2F("rise40B_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)
rise40C_vs_amp = ROOT.TH2F("rise40C_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)
rise40D_vs_amp = ROOT.TH2F("rise40D_vs_amp", "", nbins, 0, 100, nbins, 0.0, 20)

rise90_vs_rise40_ER = ROOT.TH2F("rise90_vs_rise40_ER", "", nbins, 6, 15, nbins, 1, 5)
rise90_vs_rise40_NR = ROOT.TH2F("rise90_vs_rise40_NR", "", nbins, 6, 15, nbins, 1, 5)
for i in range(len(primaryPulsesER)):
    riseER1.Fill(primaryRiseER90[i]-primaryRiseER10[i])
    riseER1_40.Fill(primaryRiseER40[i]-primaryRiseER10[i])
    rise90_vs_energy.Fill(energiesER[i], primaryRiseER90[i]-primaryRiseER10[i])
    rise40_vs_energy.Fill(energiesER[i], primaryRiseER40[i]-primaryRiseER10[i])
    rise90B_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][1],.9)-calcRise(pulsesER[i][1],.1))
    rise90C_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][2],.9)-calcRise(pulsesER[i][2],.1))
    rise90D_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][3],.9)-calcRise(pulsesER[i][3],.1))
    rise40B_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][1],.4)-calcRise(pulsesER[i][1],.1))
    rise40C_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][2],.4)-calcRise(pulsesER[i][2],.1))
    rise40D_vs_energy.Fill(energiesER[i], calcRise(pulsesER[i][3],.4)-calcRise(pulsesER[i][3],.1))
    
    rise90B_vs_amp.Fill(max(pulsesER[i][1]), calcRise(pulsesER[i][1],.9)-calcRise(pulsesER[i][1],.1))
    rise90C_vs_amp.Fill(max(pulsesER[i][2]), calcRise(pulsesER[i][2],.9)-calcRise(pulsesER[i][2],.1))
    rise90D_vs_amp.Fill(max(pulsesER[i][3]), calcRise(pulsesER[i][3],.9)-calcRise(pulsesER[i][3],.1))
    rise40B_vs_amp.Fill(max(pulsesER[i][1]), calcRise(pulsesER[i][1],.4)-calcRise(pulsesER[i][1],.1))
    rise40C_vs_amp.Fill(max(pulsesER[i][2]), calcRise(pulsesER[i][2],.4)-calcRise(pulsesER[i][2],.1))
    rise40D_vs_amp.Fill(max(pulsesER[i][3]), calcRise(pulsesER[i][3],.4)-calcRise(pulsesER[i][3],.1))
    
    rise90_vs_rise40_ER.Fill(primaryRiseER90[i]-primaryRiseER10[i], primaryRiseER40[i]-primaryRiseER10[i])
for i in range(len(primaryPulsesNR)):
    riseNR1.Fill(primaryRiseNR90[i]-primaryRiseNR10[i])
    riseNR1_40.Fill(primaryRiseNR40[i]-primaryRiseNR10[i])
    rise90_vs_energy1.Fill(energiesNR[i], primaryRiseNR90[i]-primaryRiseNR10[i])
    rise40_vs_energy1.Fill(energiesNR[i], primaryRiseNR40[i]-primaryRiseNR10[i])
    rise90_vs_rise40_NR.Fill(primaryRiseNR90[i]-primaryRiseNR10[i], primaryRiseNR40[i]-primaryRiseNR10[i])
    
for i in range(nbins):
    a = riseER1.GetBinContent(i)
    riseER.SetBinContent(i, a/len(energiesER))
    b = riseER1_40.GetBinContent(i)
    riseER_40.SetBinContent(i, b/len(energiesER))
for i in range(nbins):
    a = riseNR1.GetBinContent(i)
    riseNR.SetBinContent(i, a/len(energiesNR))
    b = riseNR1_40.GetBinContent(i)
    riseNR_40.SetBinContent(i, b/len(energiesNR))

In [12]:
# randomly selected and overlaid pulses
legend, line10, line40, line90, template_label, line10_1, line40_1, line90_1 = {}, {}, {}, {}, {}, {}, {}, {}
pulse_graphs, pulse_graphs1 = {}, {}
channel_label = {}
cPulseComparison = ROOT.TCanvas("cPulseComparison", "", 1000, 500)
nx = 3
ny = 2
cPulseComparison.Divide(nx,ny)
for k in range(nx*ny):
    i = random.randint(0,len(primaryPulsesER)-1)
    j = random.randint(0,len(primaryPulsesNR)-1)
    
    cPulseComparison.cd(k+1)
    minX = 250
    maxX = 350
    
    X = [float(i) for i in range(nSamples)]
    X = np.array(X)
    Y = np.array(primaryPulsesER[i])
    amp1 = max(Y)
    Y /= max(Y)
    pulse_graphs[k] = ROOT.TGraph(len(Y), X, Y)
    
    maxY = 1.1
    minY = -0.2
    
    pulse_graphs[k].SetLineColor(ROOT.kRed)
    pulse_graphs[k].SetMarkerColor(ROOT.kRed)
    pulse_graphs[k].SetLineWidth(2)
    pulse_graphs[k].GetXaxis().SetRangeUser(minX, maxX)
    #pulse_graphs[k].GetYaxis().SetRangeUser(minY, maxY)
    pulse_graphs[k].GetXaxis().SetTitle('us')
    pulse_graphs[k].GetYaxis().SetTitle('a.u')
    pulse_graphs[k].GetYaxis().SetTitleOffset(1.4)
    pulse_graphs[k].SetTitle("E1: "+str(energiesER[i])+" keV, rise1: "+str(round((primaryRiseER90[i]-primaryRiseER10[i]), 1))\
                           +"#mus, E2: "+str(energiesNR[j])+"keV. rise2: "+str(round((primaryRiseNR90[j]-primaryRiseNR10[j]), 1))+'#mus')
    pulse_graphs[k].Draw()
    
    line10[k] = ROOT.TLine(primaryRiseER10[i], minY, primaryRiseER10[i], maxY)
    line10[k].SetLineColor(ROOT.kBlack)
    line10[k].SetLineStyle(7)
    line10[k].SetLineWidth(2)
    line10[k].Draw("SAME")
    
    line40[k] = ROOT.TLine(primaryRiseER40[i], minY, primaryRiseER40[i], maxY)
    line40[k].SetLineColor(ROOT.kOrange)
    line40[k].SetLineStyle(7)
    line40[k].SetLineWidth(2)
    #line40[i].Draw("SAME")
    
    line90[k] = ROOT.TLine(primaryRiseER90[i], minY, primaryRiseER90[i], maxY)
    line90[k].SetLineColor(ROOT.kRed);
    line90[k].SetLineStyle(7);
    line90[k].SetLineWidth(2);
    line90[k].Draw("SAME")

    legend[k] = ROOT.TLegend(0.7,0.555,0.9,0.9)
    legend[k].AddEntry(pulse_graphs[k],"Ch"+channels[pchannelER[i]]+" pulse","l")
    legend[k].AddEntry(line10[k],"rise10","l")
    #legend[i].AddEntry(line40[k],"rise40","l")
    legend[k].AddEntry(line90[k],"rise90","l")
    legend[k].Draw("SAME")
    
    #channel_label[k] = ROOT.TLatex(minX+.5*(maxX-minX),0.9*maxY," "+channels[pchannelER[i]])
    #channel_label[k].Draw("SAME")
    
    X_shift = primaryRiseNR10[j] - primaryRiseER10[i]
    X-=X_shift
    Y = np.array(primaryPulsesNR[j])
    amp2 = max(Y)
    Y /= max(Y)
    pulse_graphs1[k] = ROOT.TGraph(len(Y), X, Y)
    
    pulse_graphs1[k].SetLineColor(ROOT.kBlue)
    pulse_graphs1[k].SetMarkerColor(ROOT.kBlue)
    pulse_graphs1[k].SetLineWidth(2)
    pulse_graphs1[k].Draw("SAME")
    
    line10_1[k] = ROOT.TLine(primaryRiseNR10[j], minY, primaryRiseNR10[j], maxY);
    line10_1[k].SetLineColor(ROOT.kBlue);
    line10_1[k].SetLineStyle(7);
    line10_1[k].SetLineWidth(2);
    #line10_1[k].Draw("SAME")
    
    line40_1[k] = ROOT.TLine(primaryRiseNR40[j], minY, primaryRiseNR40[j], maxY);
    line40_1[k].SetLineColor(ROOT.kBlue);
    line40_1[k].SetLineStyle(7);
    line40_1[k].SetLineWidth(2);
    #line40_1[i].Draw("SAME")
    
    line90_1[k] = ROOT.TLine(primaryRiseNR90[j]-X_shift, minY, primaryRiseNR90[j]-X_shift, maxY)
    line90_1[k].SetLineColor(ROOT.kBlue)
    line90_1[k].SetLineStyle(7)
    line90_1[k].SetLineWidth(2)
    line90_1[k].Draw("SAME")
    
    legend[k].AddEntry(pulse_graphs1[k],"Ch"+channels[pchannelNR[j]]+" pulse","l")
    #legend[k].AddEntry(line10_1[k],"rise10","l")
    #legend[k].AddEntry(line40_1[k],"rise40","l")
    legend[k].AddEntry(line90_1[k],"rise90","l")
    legend[k].Draw("SAME")
cPulseComparison.Draw()
cPulseComparison.SaveAs("0713_Overlaid_pulses_11.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cPulseComparison
Info in <TCanvas::Print>: png file 0713_Overlaid_pulses_11.png has been created


In [13]:
cRise = ROOT.TCanvas("cRise", "", 1000, 500)
cRise.Divide(2,1)
cRise.cd(1)
riseNR.SetStats(0)
riseNR.SetLineColor(ROOT.kBlue)
riseNR.SetLineWidth(2)
riseNR.GetXaxis().SetRangeUser(6, 18)
riseNR.GetXaxis().SetTitle('#mus (rise90)')
riseNR.GetYaxis().SetTitle('Normalized')
riseNR.GetYaxis().SetTitleOffset(1.6)
riseNR.Draw()
riseER.SetLineColor(ROOT.kRed)
riseER.SetLineWidth(2)
riseER.Draw("SAME")

legend = ROOT.TLegend(0.8,0.7,0.9,0.9)
legend.AddEntry(riseNR,"NR","l")
legend.AddEntry(riseER,"ER","l")
legend.Draw("SAME")

cRise.cd(2)
riseNR_40.SetStats(0)
riseNR_40.SetLineColor(ROOT.kBlue)
riseNR_40.SetLineWidth(2)
riseNR_40.GetXaxis().SetRangeUser(1, 5)
riseNR_40.GetXaxis().SetTitle('#mus (rise40)')
riseNR_40.GetYaxis().SetTitle('Normalized')
riseNR_40.GetYaxis().SetTitleOffset(1.6)
riseNR_40.Draw()
riseER_40.SetLineColor(ROOT.kRed)
riseER_40.SetLineWidth(2)
riseER_40.Draw("SAME")

legend1 = ROOT.TLegend(0.8,0.7,0.9,0.9)
legend1.AddEntry(riseNR,"NR","l")
legend1.AddEntry(riseER,"ER","l")
legend1.Draw("SAME")
cRise.Draw()
#cRise.SaveAs("rise_time_comparison_ER_NR.png")

###### 

In [14]:
cRise_vs_energy = ROOT.TCanvas("cRise_vs_energy", "", 1000, 500)
cRise_vs_energy.Divide(2,1)
cRise_vs_energy.cd(1)
rise90_vs_energy.SetStats(0)
rise90_vs_energy.SetMarkerColor(ROOT.kRed)
rise90_vs_energy.SetMarkerStyle(8)
rise90_vs_energy.SetMarkerSize(1)
rise90_vs_energy.GetYaxis().SetRangeUser(6, 18)
rise90_vs_energy.GetXaxis().SetRangeUser(0.6, 3.5)
rise90_vs_energy.GetYaxis().SetTitle('#mus (rise90)')
rise90_vs_energy.GetXaxis().SetTitle('keV')
rise90_vs_energy.GetYaxis().SetTitleOffset(1.6)
rise90_vs_energy.Draw()
rise90_vs_energy1.SetMarkerColor(ROOT.kBlue)
rise90_vs_energy1.SetMarkerStyle(8)
rise90_vs_energy1.SetMarkerSize(1)
rise90_vs_energy1.Draw("SAME")

legend = ROOT.TLegend(0.8,0.7,0.9,0.9)
legend.AddEntry(rise90_vs_energy1,"NR","p")
legend.AddEntry(rise90_vs_energy,"ER","p")
legend.Draw("SAME")

cRise_vs_energy.cd(2)
rise40_vs_energy.SetStats(0)
rise40_vs_energy.SetMarkerColor(ROOT.kRed)
rise40_vs_energy.SetMarkerStyle(8)
rise40_vs_energy.SetMarkerSize(1)
rise40_vs_energy.GetYaxis().SetRangeUser(1, 5)
rise40_vs_energy.GetXaxis().SetRangeUser(0.6, 3.5)
rise40_vs_energy.GetYaxis().SetTitle('#mus (rise40)')
rise40_vs_energy.GetXaxis().SetTitle('keV')
rise40_vs_energy.GetYaxis().SetTitleOffset(1.6)
rise40_vs_energy.Draw()
rise40_vs_energy1.SetMarkerColor(ROOT.kBlue)
rise40_vs_energy1.SetMarkerStyle(8)
rise40_vs_energy1.SetMarkerSize(1)
rise40_vs_energy1.Draw("SAME")

legend1 = ROOT.TLegend(0.8,0.7,0.9,0.9)
legend1.AddEntry(rise40_vs_energy1,"NR","p")
legend1.AddEntry(rise40_vs_energy,"ER","p")
legend1.Draw("SAME")
cRise_vs_energy.Draw()
#cRise_vs_energy.SaveAs("rise_vs_energy_ER_NR.png")

In [15]:
markersize = .7
cRise_vs_energy_channels = ROOT.TCanvas("cRise_vs_energy_channels", "", 1000, 500)
cRise_vs_energy_channels.Divide(2,1)
cRise_vs_energy_channels.cd(1)
rise90B_vs_energy.SetStats(0)
rise90B_vs_energy.SetMarkerColor(ROOT.kRed)
rise90B_vs_energy.SetMarkerStyle(8)
rise90B_vs_energy.SetMarkerSize(markersize)
rise90B_vs_energy.GetYaxis().SetRangeUser(6, 20)
rise90B_vs_energy.GetYaxis().SetTitle('#mus (rise90)')
rise90B_vs_energy.GetXaxis().SetTitle('keV')
rise90B_vs_energy.GetYaxis().SetTitleOffset(1.4)
rise90B_vs_energy.Draw()
rise90C_vs_energy.SetMarkerColor(ROOT.kBlue)
rise90C_vs_energy.SetMarkerStyle(8)
rise90C_vs_energy.SetMarkerSize(markersize)
rise90C_vs_energy.Draw("SAME")
rise90D_vs_energy.SetMarkerColor(ROOT.kGreen)
rise90D_vs_energy.SetMarkerStyle(8)
rise90D_vs_energy.SetMarkerSize(markersize)
rise90D_vs_energy.Draw("SAME")

legend = ROOT.TLegend(0.7,0.555,0.9,0.9)
legend.AddEntry(rise90B_vs_energy,"Channel B","p")
legend.AddEntry(rise90C_vs_energy,"Channel C","p")
legend.AddEntry(rise90D_vs_energy,"Channel D","p")
legend.Draw("SAME")

cRise_vs_energy_channels.cd(2)
rise90B_vs_amp.SetStats(0)
rise90B_vs_amp.SetMarkerColor(ROOT.kRed)
rise90B_vs_amp.SetMarkerStyle(8)
rise90B_vs_amp.SetMarkerSize(markersize)
rise90B_vs_amp.GetYaxis().SetRangeUser(6, 20)
rise90B_vs_amp.GetYaxis().SetTitle('#mus (rise90)')
rise90B_vs_amp.GetXaxis().SetTitle('a.u.')
rise90B_vs_amp.GetYaxis().SetTitleOffset(1.4)
rise90B_vs_amp.Draw()
rise90C_vs_amp.SetMarkerColor(ROOT.kBlue)
rise90C_vs_amp.SetMarkerStyle(8)
rise90C_vs_amp.SetMarkerSize(markersize)
rise90C_vs_amp.Draw("SAME")
rise90D_vs_amp.SetMarkerColor(ROOT.kGreen)
rise90D_vs_amp.SetMarkerStyle(8)
rise90D_vs_amp.SetMarkerSize(markersize)
rise90D_vs_amp.Draw("SAME")

legend1 = ROOT.TLegend(0.7,0.555,0.9,0.9)
legend1.AddEntry(rise90B_vs_amp,"Channel B","p")
legend1.AddEntry(rise90C_vs_amp,"Channel C","p")
legend1.AddEntry(rise90D_vs_amp,"Channel D","p")
legend1.Draw("SAME")
cRise_vs_energy_channels.Draw()
#cRise_vs_energy_channels.SaveAs("rise90_v_channelsBCD_ER.png")

In [16]:
markersize = .7
cRise_vs_energy_channels1 = ROOT.TCanvas("cRise_vs_energy_channels1", "", 1000, 500)
cRise_vs_energy_channels1.Divide(2,1)
cRise_vs_energy_channels1.cd(1)
rise40B_vs_energy.SetStats(0)
rise40B_vs_energy.SetMarkerColor(ROOT.kRed)
rise40B_vs_energy.SetMarkerStyle(8)
rise40B_vs_energy.SetMarkerSize(markersize)
rise40B_vs_energy.GetYaxis().SetRangeUser(1, 10)
rise40B_vs_energy.GetYaxis().SetTitle('#mus (rise40)')
rise40B_vs_energy.GetXaxis().SetTitle('keV')
rise40B_vs_energy.GetYaxis().SetTitleOffset(1.4)
rise40B_vs_energy.Draw()
rise40C_vs_energy.SetMarkerColor(ROOT.kBlue)
rise40C_vs_energy.SetMarkerStyle(8)
rise40C_vs_energy.SetMarkerSize(markersize)
rise40C_vs_energy.Draw("SAME")
rise40D_vs_energy.SetMarkerColor(ROOT.kGreen)
rise40D_vs_energy.SetMarkerStyle(8)
rise40D_vs_energy.SetMarkerSize(markersize)
rise40D_vs_energy.Draw("SAME")

legend = ROOT.TLegend(0.7,0.555,0.9,0.9)
legend.AddEntry(rise40B_vs_energy,"Channel B","p")
legend.AddEntry(rise40C_vs_energy,"Channel C","p")
legend.AddEntry(rise40D_vs_energy,"Channel D","p")
legend.Draw("SAME")

cRise_vs_energy_channels1.cd(2)
rise40B_vs_amp.SetStats(0)
rise40B_vs_amp.SetMarkerColor(ROOT.kRed)
rise40B_vs_amp.SetMarkerStyle(8)
rise40B_vs_amp.SetMarkerSize(markersize)
rise40B_vs_amp.GetYaxis().SetRangeUser(1, 10)
rise40B_vs_amp.GetYaxis().SetTitle('#mus (rise40)')
rise40B_vs_amp.GetXaxis().SetTitle('a.u.')
rise40B_vs_amp.GetYaxis().SetTitleOffset(1.4)
rise40B_vs_amp.Draw()
rise40C_vs_amp.SetMarkerColor(ROOT.kBlue)
rise40C_vs_amp.SetMarkerStyle(8)
rise40C_vs_amp.SetMarkerSize(markersize)
rise40C_vs_amp.Draw("SAME")
rise40D_vs_amp.SetMarkerColor(ROOT.kGreen)
rise40D_vs_amp.SetMarkerStyle(8)
rise40D_vs_amp.SetMarkerSize(markersize)
rise40D_vs_amp.Draw("SAME")

legend1 = ROOT.TLegend(0.7,0.555,0.9,0.9)
legend1.AddEntry(rise90B_vs_amp,"Channel B","p")
legend1.AddEntry(rise90C_vs_amp,"Channel C","p")
legend1.AddEntry(rise90D_vs_amp,"Channel D","p")
legend1.Draw("SAME")
cRise_vs_energy_channels1.Draw()
#cRise_vs_energy_channels1.SaveAs("rise40_v_channelsBCD_ER.png")

In [17]:
cRise90_vs_rise40 = ROOT.TCanvas("cRise90_vs_rise40", "", 500, 500)
cRise90_vs_rise40.cd()
rise90_vs_rise40_ER.SetStats(0)
rise90_vs_rise40_ER.SetMarkerColor(ROOT.kRed)
rise90_vs_rise40_ER.SetMarkerStyle(8)
rise90_vs_rise40_ER.SetMarkerSize(1)
rise90_vs_rise40_ER.GetYaxis().SetRangeUser(1, 5)
rise90_vs_rise40_ER.GetXaxis().SetRangeUser(0.6, 3.5)
rise90_vs_rise40_ER.GetYaxis().SetTitle('#mus (rise40)')
rise90_vs_rise40_ER.GetXaxis().SetTitle('#mus (rise90)')
rise90_vs_rise40_ER.GetYaxis().SetTitleOffset(1.6)
rise90_vs_rise40_ER.Draw()
rise90_vs_rise40_NR.SetMarkerColor(ROOT.kBlue)
rise90_vs_rise40_NR.SetMarkerStyle(8)
rise90_vs_rise40_NR.SetMarkerSize(1)
rise90_vs_rise40_NR.Draw("SAME")

legend1 = ROOT.TLegend(0.8,0.7,0.9,0.9)
legend1.AddEntry(rise90_vs_rise40_NR,"NR","p")
legend1.AddEntry(rise90_vs_rise40_ER,"ER","p")
legend1.Draw("SAME")
cRise90_vs_rise40.Draw()
#cRise90_vs_rise40.SaveAs("rise90_vs_rise40.png")